In [6]:
%load_ext autoreload
%autoreload 2

import streamlit as st
from catdel.components import misc
from catdel.config import Config
from catdel import process
from catdel.projection import Projection
from catdel.grid_processors import get_grid_bounds_geo, get_grid_center_geo
import folium
from IPython.display import display
from pysheds.grid import Grid


config = Config()

# Create a map object centered on Portland, Oregon

# Add a marker


# Display the map in the Jupyter notebook



fn = '/home/ashkan/Downloads/DEM-1m.tif'
grid, dem = process.load_grid_and_data(fn, config.proj.pyproj)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/ashkan/projects/catdel/.venv/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/ashkan/projects/catdel/.venv/lib/python3.10/site-packages/pysheds/io.py:134: UserWarning: No `nodata` value detected. Defaulting to 0.
  warnings.warn('No `nodata` value detected. Defaulting to 0.')


In [7]:
#process.delin(dem, grid, config)


# Fill pits in DEM
pit_filled_dem = grid.fill_pits(dem)
# Fill depressions in DEM
flooded_dem = grid.fill_depressions(pit_filled_dem)
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
fdir = grid.flowdir(inflated_dem, dirmap=dirmap)
acc = grid.accumulation(fdir, dirmap=dirmap)

# Snap pour point to high accumulation cell
lat, lng = -38.1923, 140.2318

lat_snap, lng_snap = grid.snap_to_mask(acc > config.snap_thr, (lat, lng))


# Delineate the catchment
catch = grid.catchment(x=lat_snap,
                    y=lng_snap,
                    fdir=fdir,
                    dirmap=dirmap,
                    xytype='coordinate')

branches = grid.extract_river_network(fdir,
                                        acc > config.acc_thr,
                                        dirmap=dirmap)


out = {'catch': catch, 'branches':branches}


